# COMP 551 - Mini-project 3
Group 63

In [1]:
import keras
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import scipy.sparse as sparse
import pandas as pd
import seaborn as sns

## Load and preprocess the data

- Load the raw data from Keras.
- augment data
- Vectorize 28*28 pictures to 1D vector.
- Normalize the intensity of the pixel.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(x_train.shape,y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


### Creating augmented dataset

In [3]:
#creating augmented dataset
x_train_augmented = [image for image in x_train]
y_train_augmented = [image for image in y_train]

In [4]:
from scipy.ndimage.interpolation import shift
def shift_images(image, dx, dy):
  image = image.reshape((28,28))
  shifted_image = shift(image, [dy,dx], cval=0, mode="constant")
  #return shifted_image.reshape([-1])
  return shifted_image


In [5]:
for dx, dy in ((1,0), (-1,0),(0,1),(0,-1)):
  for image, label in zip(x_train, y_train):
    x_train_augmented.append(shift_images(image, dx, dy))
    y_train_augmented.append(label)

In [6]:
shuffle_idx = np.random.permutation(len(x_train_augmented))

In [7]:
x_train_augmented = np.array(x_train_augmented)[shuffle_idx]
y_train_augmented = np.array(y_train_augmented)[shuffle_idx]

In [8]:
x_train_augmented.shape

(300000, 28, 28)

In [9]:
x_train_augmented1 = np.reshape(x_train_augmented, (-1, 784)).astype('float32')
x_test = np.reshape(x_test, (-1, 784)).astype('float32')
print(x_train_augmented1.shape, x_test.shape)

(300000, 784) (10000, 784)


In [10]:
print('Intensity before normalization:', np.amin(x_train_augmented1), np.amax(x_train_augmented1))
x_train_augmented1, x_test = x_train_augmented1/255.0, x_test/255.0
print('Intensity after normalization:', np.amin(x_train_augmented1), np.amax(x_train_augmented1))

Intensity before normalization: 0.0 255.0
Intensity after normalization: 0.0 1.0


In [11]:
y_train_augmented = keras.utils.to_categorical(y_train_augmented)
y_test = keras.utils.to_categorical(y_test)

## Multilayer perceptron implementation
### Build the network
Our task is a multiclass classification.The cost function will be the multi-class cross-entropy loss. We will use the following architecture:
- output layer = softmax activation
- hidden layers (0, 1 or 2): 128 units tanh activation function (it could also be ReLu or logistic function)

Build the activation function

In [12]:
# logistic/sigmoid
logistic = lambda z: 1./ (1 + np.exp(-z))

# softmax
eps=1e-8
def softmax(z):
    logits = z - np.max(z) # for numerical stability
    sum_logits = np.sum(np.exp(logits), axis=1) +eps
    softmax = np.exp(logits)/sum_logits[:,None] 
    return softmax

# relu
relu = lambda z: np.maximum(0,z)

# derivatives of relu (formula from backprop slides)
def relu_dv(q):
  q[q<=0] = 0
  q[q>0] = 1
  return q

Create the MLP architecture for no hidden layer

In [13]:
 # for no hidden layer 
 class MLP_none:
    
    def __init__(self, M=128):
        self.M = M
            
    def fit(self, x, y, optimizer):
        N = x.shape[0]
        C = y.shape[1] # number of classes
        x = np.column_stack([x,np.ones(N)])
        D = x.shape[1]
        def gradient(x, y, w): 
            # forward pass
            yh = softmax(np.dot(x, w))#N x C
            # backward pass => gradient formula from class dw = (yh-y)*x
            dy = yh - y #N x C
            dw = np.dot(x.T, dy)/N #D x C
            return dw
        
        # initialize the parameters with values in the standard normal distribution and scaled to be low
        w = np.random.randn(D,C) * .01 #D x C
        self.params = optimizer.run(gradient, x, y, w)
        return self
    
    def predict(self, x):
        w = self.params
        Nt = x.shape[0]
        x = np.column_stack([x,np.ones(Nt)])
        yh = softmax(np.dot(x, w))#N x C
        return yh

Create the MLP architecture for one hidden layer with ReLu activation function

In [14]:
 # for 1 hidden layer with relu activation
 class MLP_relu:
    
    def __init__(self, M = 128):
        self.M = M
            
    def fit(self, x, y, optimizer):
        N = x.shape[0]
        C = y.shape[1] # number of classes
        # add bias to the input layer
        x = np.column_stack([x,np.ones(N)])
        D = x.shape[1]
        def gradient(x, y, params):
            v, w = params
            # forward pass
            q = np.dot(x, v)
            z = relu(q) #N x M
            yh = softmax(np.dot(z, w))#N x C
            # backward pass => gradient formula adapted from class dw = (yh-y)*z, dv = (yh-y)*w*deriv_relu(q)*x
            dy = yh - y #N x C
            dw = np.dot(z.T, dy)/N #M x C
            dz = np.dot(dy, w.T) #N x M
            dq = relu_dv(q)
            dv = np.dot(x.T, dz * dq)/N #D x M
            dparams = [dv, dw]
            return dparams
        
        # initialize the parameters with values in the standard normal distribution and scaled to be low
        w = np.random.randn(self.M,C) * .01 #M x C
        v = np.random.randn(D,self.M) * .01 #D x M
        params0 = [v,w]

        # run the mini-batch gradient descent to update the parameters
        self.params = optimizer.run(gradient, x, y, params0)
        return self
    
    def predict(self, x):
        v, w = self.params
        # add bias to the input layer
        Nt = x.shape[0]
        x = np.column_stack([x,np.ones(Nt)])
        # forward pass only using updated parameters
        z = relu(np.dot(x, v)) #N x M
        yh = softmax(np.dot(z, w))#N x C
        return yh

Create the MLP architecture for two hidden layer with ReLu activation function

In [15]:
# for 2 hidden layer with relu activation
class MLP2layer_relu:
    
    def __init__(self, M = 128):
        self.M = M
            
    def fit(self, x, y, optimizer):
        N = x.shape[0]
        C = y.shape[1] # number of classes
        # add bias to the input layer
        x = np.column_stack([x,np.ones(N)*0.1])
        D = x.shape[1]
        def gradient(x, y, params):
            v, w, u = params
            # forward pass
            n = x.shape[0]
            b = np.ones((n,1))*0.1

            q1 = np.dot(x, v) #np.column_stack([np.dot(x, v),np.ones(N)*0.1]) #trying adding bias here
            z1 = relu(np.hstack((q1,b))) #N x M want to column stack to add bias here
            q2 = np.dot(z1,w) #np.column_stack([np.dot(z1,w),np.ones(N)*0.1]) #trying adding bias here
            z2 = relu(np.hstack((q2,b)))
            yh = softmax(np.dot(z2, u))#N x C
            # backward pass => gradient formula adapted from class dw = (yh-y)*z, dv = (yh-y)*w*deriv_relu(q)*x
            
            dy = yh - y #N x C
            
            du = np.dot(z2.T,dy)/N
            
            #print(dz2.shape)  
            dz2 = np.dot(dy,u.T)
            dz2 = np.delete(dz2, -1, axis=1)
            dq2 = relu_dv(q2)
            #print(dz2.T * dq2)
            dw = np.dot(z1.T, dz2 * dq2)/N #M x C
            dz1 = np.dot(dz2, w.T) #N x M
            dz1 = np.delete(dz1,-1,axis=1)
            dq1 = relu_dv(q1)
            dv = np.dot(x.T, dz1 * dq1)/N #D x M
            dparams = [dv, dw, du]
            return dparams
        
        # initialize the parameters with values in the standard normal distribution and scaled to be low
        u = np.random.randn(self.M+1,C) * 0.1 #M x C
        w = np.random.randn(self.M+1,self.M) * .01 #M x M
        v = np.random.randn(D,self.M) * .01 #D x M
        
        params0 = [v,w,u]

        # run the mini-batch gradient descent to update the parameters
        self.params = optimizer.run(gradient, x, y, params0)
        return self
    
    def predict(self, x):
        v, w, u = self.params
        # add bias to the input layer
        Nt = x.shape[0]
        x = np.column_stack([x,np.ones(Nt)*0.1])
        b1 = np.ones((Nt,1))*0.1
     
        # forward pass only using updated parameters

        q1 = np.dot(x,v)
        z1 = relu(np.hstack((q1,b1)))
        q2 = np.dot(z1,w)
        z2 = relu(np.hstack((q2,b1)))
        yh = softmax(np.dot(z2, u))#N x C
        return yh

### Implement the cost and accuracy function

In [16]:
# Softmax cross entropy 
def logsumexp(Z):                                                # dimension C x N
    Zmax = np.max(Z,axis=0)[None,:]                              # max over C
    log_sum_exp = Zmax + np.log(np.sum(np.exp(Z - Zmax), axis=0))
    return log_sum_exp

# cost for relu activation
def cost_relu(x, #N x D 
         y, #N x C 
         w, #M x C 
         v, #D x M
        ):
  q = np.dot(x, v) #N x M
  z = relu(q) #N x M
  u = np.dot(z, w) #N x C
  yh = softmax(u)
  nll = - np.mean(np.sum(u*y, 1) - logsumexp(u)) 
  return nll

# Accuracy
def evaluate_acc(y, yh):
  y_pred = np.argmax(yh,axis=1)
  accuracy = np.count_nonzero(y_pred == np.argmax(y,axis=1))/y.shape[0]
  return accuracy

### Implement the optimizer

We will use a mini-batch gradient-descent algorithm.

In [17]:
def create_mini_batch(x, y, batch_size): 
    D = x.shape[1]
    data = np.hstack((x, y))
    np.random.shuffle(data)
    mini = data[:batch_size,:]                                                    
    x_mini = mini[:,:D]
    y_mini = mini[:,D:]
    return x_mini, y_mini

In [18]:
class GradientDescent:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8, batch_size=100):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.batch_size = batch_size
        
    def run(self, gradient_fn, x, y, params):
        norms = np.array([np.inf])
        t = 1
        while np.any(norms > self.epsilon) and t < self.max_iters:
            # create mini-batch
            x_mini, y_mini = create_mini_batch(x, y, self.batch_size)
            # calculate gradient for the mini-batch
            grad = gradient_fn(x_mini, y_mini, params)
            # update v and dw
            for p in range(len(params)):
                params[p] -= self.learning_rate * grad[p]
            t += 1
            norms = np.array([np.linalg.norm(g) for g in grad])
        return params


In [19]:
class GradientDescent_none:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8, batch_size=100):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.batch_size = batch_size
        
    def run(self, gradient_fn, x, y, params):
        norms = np.array([np.inf])
        t = 1
        while np.any(norms > self.epsilon) and t < self.max_iters:
            x_mini, y_mini = create_mini_batch(x, y, self.batch_size)
            grad = gradient_fn(x_mini, y_mini, params)
            params -= self.learning_rate * grad
            t += 1
            norms = np.array([np.linalg.norm(g) for g in grad])
        return params


## Run the experiments

Test MLP with no hidden layer

In [23]:
model = MLP_none()
optimizer = GradientDescent_none(learning_rate=.1, max_iters=20000)

model.fit(x_train_augmented1, y_train_augmented, optimizer)
yh = model.predict(x_test) 
print(yh.shape)
print(yh[:3,])
accuracy = evaluate_acc(y_test, yh)
print(f'Accuracy is {accuracy*100:.1f}.')

(10000, 10)
[[0.09966634 0.05674332 0.08324794 0.08729533 0.09625264 0.07304213
  0.0706631  0.21512254 0.0894713  0.12849534]
 [0.11673741 0.10468862 0.13776297 0.13645928 0.05004836 0.1013022
  0.12139154 0.05088396 0.12218948 0.05853616]
 [0.06807997 0.18914192 0.10497551 0.10542814 0.07864706 0.0808471
  0.09236405 0.08733863 0.1044884  0.0886892 ]]
Accuracy is 75.4.


Test MLP with one hidden layer for ReLu activation function

In [31]:
model = MLP_relu()
optimizer = GradientDescent(learning_rate=2, max_iters=20000)

model.fit(x_train_augmented1, y_train_augmented, optimizer)
yh = model.predict(x_test) 
print(yh.shape)
print(yh[:3,])
accuracy = evaluate_acc(y_test, yh)
print(f'Accuracy is {accuracy*100:.1f}.')

(10000, 10)
[[2.09509455e-05 4.09279205e-09 3.73911328e-04 4.95574179e-03
  6.87811722e-10 1.60439263e-05 3.31668917e-11 9.92348567e-01
  9.11942778e-06 4.65373321e-05]
 [5.92817122e-04 1.01666044e-03 8.82237191e-01 2.46249257e-02
  2.66581757e-07 1.29132905e-03 8.39740985e-03 2.17716040e-08
  9.91867270e-03 4.35509703e-08]
 [9.99548098e-07 6.55654999e-01 2.52778219e-03 3.71767921e-04
  1.15981043e-03 5.84105696e-05 3.82219255e-03 2.43303196e-03
  7.28347919e-04 6.13814982e-05]]
Accuracy is 93.9.


Test MLP with two hidden layer for ReLu activation function

In [26]:
model = MLP2layer_relu()
optimizer = GradientDescent(learning_rate=2, max_iters=20000)

model.fit(x_train_augmented1, y_train_augmented, optimizer)
yh = model.predict(x_test) 
print(yh.shape)
print(yh[:3,])
accuracy = evaluate_acc(y_test, yh)
print(f'Accuracy is {accuracy*100:.1f}.')

(10000, 10)
[[1.03667292e-03 7.44680901e-04 7.57676393e-04 9.08588440e-03
 2.91209439e-03 1.53216466e-03 2.99343052e-05 9.14802823e-01
 4.83905104e-03 6.42567058e-02]
[4.24931465e-02 4.42952453e-03 6.66883630e-01 8.45419289e-02
 1.82487462e-04 4.93942852e-02 9.82304308e-02 1.67124401e-05
 5.37333287e-02 8.86373658e-05]
[3.56114690e-04 8.94878568e-01 3.05090741e-02 1.80016065e-02
 3.51352037e-03 2.40707183e-03 7.95288951e-03 9.57534981e-03
 2.58602401e-02 6.94000582e-03]]
Accuracy is 96.7.


In [22]:
def confusion_matrix(y, yh):
    n_classes = y.shape[1]
    c_matrix = np.zeros((n_classes, n_classes))
    for c1 in range(n_classes):
        for c2 in range(n_classes):
        #(y==c1)*(yh==c2) is 1 when both conditions are true or 0
            c_matrix[c1, c2] = np.sum((np.argmax(y,axis=1)==c1)*(np.argmax(yh,axis=1)==c2))
    return c_matrix


In [23]:
cmat = confusion_matrix(y_test, yh)
np.set_printoptions(suppress=True)
print(cmat)

[[ 939.    0.    4.    2.    0.   25.    6.    1.    3.    0.]
 [   0. 1104.    5.    5.    0.    0.    3.    0.   17.    1.]
 [  22.   36.  856.   23.   21.    2.   28.    9.   32.    3.]
 [   5.    5.   26.  878.    1.   36.    1.   21.   30.    7.]
 [   2.    8.    4.    0.  824.    0.   24.    1.    4.  115.]
 [  30.    4.   12.   97.   19.  603.   38.   14.   62.   13.]
 [  27.    3.   19.    1.   19.   15.  871.    0.    3.    0.]
 [   4.   41.   21.    1.    7.    1.    0.  882.    7.   64.]
 [   7.   22.   12.   44.   14.   69.   24.   13.  744.   25.]
 [   9.    9.    4.   10.  130.   15.    0.   53.   12.  767.]]
